In [5]:
# LSTM 模型 與 新聞情緒模型： 晚期融合
import pandas as pd

# 1. 讀取兩個模型輸出結果
stock_data = pd.read_csv("week11/predicted_prices_2008-09-09-2015-12-31.csv")
news_data = pd.read_csv("week11/daily_sentiment_score_2008-06-08-2016-07-01.csv")

stock_data = pd.DataFrame(stock_data)
news_data = pd.DataFrame(news_data)

print(stock_data.head())
print("--------------------------------")
print(news_data.head())

         Date  Predicted_Close
0  2008-09-09        11293.779
1  2008-09-10        11461.185
2  2008-09-11        11392.132
3  2008-09-12        11339.706
4  2008-09-15        11423.463
--------------------------------
         Date  Weighted_sentiment_by_day
0  2008-06-08                  89.328063
1  2008-06-09                  97.628458
2  2008-06-10                  97.628458
3  2008-06-11                 100.000000
4  2008-06-12                  96.837945


In [6]:
same_dates = stock_data['Date'].isin(news_data['Date']).index
stock_data = stock_data.iloc[same_dates]
news_data = news_data.iloc[same_dates]


In [7]:
from sklearn.linear_model import LinearRegression
import numpy as np

# 晚期融合： 使用 情緒模型 調整 預測股價 (調整幅度不超過±5%)
emotion_model = LinearRegression()
emotion_model.fit(np.array(news_data['Weighted_sentiment_by_day']).reshape(-1, 1), np.array(stock_data['Predicted_Close']))

# 使用情緒分數微調預測股價
def apply_adjustment_limits(adjustments, limit=0.05):
    return np.clip(adjustments, -limit, limit)

adjustments = emotion_model.predict(np.array(news_data['Weighted_sentiment_by_day']).reshape(-1, 1))
adjusted_stock_prices = np.array(stock_data['Predicted_Close']) + apply_adjustment_limits(adjustments)



In [8]:
adjusted_stock_prices = pd.DataFrame(adjusted_stock_prices, columns=['Adjusted_Close'])

df = pd.concat([stock_data, adjusted_stock_prices], axis=1)
df.head()


,Date,Predicted_Close,Adjusted_Close
0,2008-09-09,11293.779,11293.829
1,2008-09-10,11461.185,11461.235
2,2008-09-11,11392.132,11392.182
3,2008-09-12,11339.706,11339.756
4,2008-09-15,11423.463,11423.513


In [18]:
output = pd.DataFrame()
output["Date"] = df["Date"]
output["Final_Predicted_Price"] = df["Adjusted_Close"]

# 存檔
output.to_csv(f"week11/final_stock_prices_{output['Date'].min()}-{output['Date'].max()}.csv", index=False)


print(output.head())

         Date  Final_Predicted_Price
0  2008-09-09              11293.829
1  2008-09-10              11461.235
2  2008-09-11              11392.182
3  2008-09-12              11339.756
4  2008-09-15              11423.513
